In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType

In [2]:
# running local spark
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.driver.memory", "12g")\
    .appName("neighborhoods_and_city") \
    .getOrCreate()

sc = spark.sparkContext

24/04/21 20:12:23 WARN Utils: Your hostname, Kun-Mac.local resolves to a loopback address: 127.0.0.1; using 172.20.23.178 instead (on interface en0)
24/04/21 20:12:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 20:12:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print("Using Apache Spark Version", spark.version)
web_ui_url = sc.uiWebUrl
print(f"Spark UI is available at: {web_ui_url}")

Using Apache Spark Version 3.5.0
Spark UI is available at: http://172.20.23.178:4040


In [4]:
listings = spark.read.option("header", "true") \
                    .option("delimiter", ",") \
                    .option("inferSchema", "true") \
                    .option("multiLine", "true")\
                    .option("escape", "\"")\
                    .csv("../Data/listings.csv")
listings = listings.select(col('id'),col('neighbourhood_cleansed'))

In [5]:
calendar = spark.read.option("header", "true") \
                    .option("delimiter", ",") \
                    .option("inferSchema", "true") \
                    .option("multiLine", "true")\
                    .option("escape", "\"")\
                    .csv("../Data/calendar.csv")
calendar = calendar.drop(*['minimum_nights','maximum_nights'])

24/04/21 20:12:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
def money_to_float(money_str):
    if money_str is None:
        return None
    else:
        cleaned_str = money_str[1:].replace(',', '')
        return float(cleaned_str)
spark.udf.register("money_to_float_udf", money_to_float, FloatType())

<function __main__.money_to_float(money_str)>

In [7]:
listings.createOrReplaceTempView('listings')
calendar.createOrReplaceTempView('calendar')
sql = '''
with neighbourhood as ( 
select
    id
    ,neighbourhood_cleansed
from listings
),
calendar_cleaned as ( 
select
    listing_id
    ,date
    ,if(available='t',1,0) as is_available
    ,coalesce(money_to_float_udf(adjusted_price),money_to_float_udf(price)) as price
    -- ,CAST(regexp_replace(price, '[^0-9\.]', '') AS FLOAT) AS price
    ,state
    ,city
    ,data_date as data_download_date
from calendar
)

select
    state
    ,city
    ,neighbourhood_cleansed
    ,date
    ,sum(is_available)/count(is_available) as occupancy_rate
    ,avg(price) as avg_price
from calendar_cleaned t1
left join neighbourhood t2
    on t1.listing_id = t2.id
group by 
    state
    ,city
    ,neighbourhood_cleansed
    ,date
'''
result_neighbourhood = spark.sql(sql)

24/04/21 20:14:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
print(result_neighbourhood.count())
result_neighbourhood.limit(5).toPandas()

24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:15:19 WARN RowBasedKeyValueBatch: Calling spill() on

577080


24/04/21 20:18:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/21 20:18:38 WARN RowBasedKeyValueBatch: Calling spill() on

,state,city,neighbourhood_cleansed,date,occupancy_rate,avg_price
0,ny,new-york-city,Flatbush,2024-03-16,0.442913,125.580709
1,ny,new-york-city,Flatbush,2024-09-15,0.427165,125.580709
2,ny,new-york-city,Flatbush,2024-11-23,0.336614,125.580709
3,ny,new-york-city,East Village,2024-02-17,0.234043,241.274746
4,ny,new-york-city,East Village,2024-04-23,0.307123,241.274746


# insert into Database (WIP)